<a href="https://colab.research.google.com/github/poojaneuusa/NLP-Projects/blob/main/Quora_Questions_Suggester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U sentence-transformers
import torch
from sentence_transformers import SentenceTransformer, util
import os
import time
import pandas as pd
import requests

torch.__version__

url = 'http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open('quora_duplicate_questions.tsv', 'wb') as file:
        file.write(response.content)
    df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t', nrows=20000)
    print(df.shape)
else:
    print(f"Failed to fetch data: {response.status_code}")

df.head()

corpus_senteces = list(set(df['question1'].tolist() + df['question2'].tolist()))
len(corpus_senteces)

model = SentenceTransformer('quora-distilbert-multilingual')

corpus_embeddings = model.encode(corpus_senteces, show_progress_bar=True, convert_to_tensor=True)

model._target_device

corpus_embeddings = corpus_embeddings.to(model._target_device)

while True:
  query = input("Please enter a question: ")

  if query == 'n':
    break

  question_embedding = model.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=5)
  hits = hits[0]
  print(hits)

  for hit in hits:
    print(hit['score'], ": ", corpus_senteces[hit['corpus_id']])

  print('\n\n\n\n')


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
(20000, 6)


Batches:   0%|          | 0/1181 [00:00<?, ?it/s]

Please enter a question: What is Python?
[{'corpus_id': 33249, 'score': 0.9740513563156128}, {'corpus_id': 1958, 'score': 0.9668587446212769}, {'corpus_id': 30760, 'score': 0.9650131464004517}, {'corpus_id': 26269, 'score': 0.9578346014022827}, {'corpus_id': 6963, 'score': 0.9539889693260193}]
0.9740513563156128 :  What is the difference between () and [] in Python?
0.9668587446212769 :  What are the differences between Python 1, 2, 3?
0.9650131464004517 :  What is the difference between /= and //= in Python?
0.9578346014022827 :  What's the best MongoDB ORM for Python?
0.9539889693260193 :  Which implementation of Python does Google use?





Please enter a question: How can i start with ML?
[{'corpus_id': 27939, 'score': 0.9046310186386108}, {'corpus_id': 28547, 'score': 0.9018148183822632}, {'corpus_id': 1548, 'score': 0.8984736204147339}, {'corpus_id': 24961, 'score': 0.8957906365394592}, {'corpus_id': 14644, 'score': 0.8950694799423218}]
0.9046310186386108 :  How to learn MATLAB?
